# Lesson 3- Calculating a Stock's Beta and Alpha

In this example, we will retrieve historical stock data from <b><i>Yahoo! Finance</i></b> and calculate a stock's beta and alpha using Python.

Recall that a stock's beta is a measure of non-diversifiable (or systematic) risk. To estimate a stock's beta, we make use of the Capital Asset Pricing Model (<b>CAPM</b>):

$$\begin{eqnarray}
E(R_i) = R_f + \beta_i(E(R_M) - R_f)  \\
\end{eqnarray}$$

$E(R_i)$ is the expected return for stock $i$, $R_f$ is the risk-free rate, $E(R_M)$ is the return of the market portfolio, and $\beta_i$ is the systematic risk, or the tendency of a security's returns to respond to swings in the overall market.

To estimate $\beta_i$ for a stock, we can regress the excess returns of a stock on the excess returns of the market. The intercept in this case will be a stock's $\alpha$. 

$$\begin{equation}
R_i - R_f = \alpha_i + \beta_i(R_M - R_f) + \epsilon_i \\
\end{equation}$$

Generally:
- $\beta_i > 1$: The stock moves in the same direction as the market, but at a greater magnitude.
- $\beta_i = 1$: The stock moves in the same direction and magnitude as the market.
- $0 < \beta_i < 1$: The stock moves in the same direction as the market, but with a lower magnitude.
- $\beta_i = 0$: The stock's movement is uncorrelated with the market.
- $-1 < \beta_i < 0$: The stock moves in the opposite direction as the market, but with a lower magnitude.
- $\beta_i < -1$: The stock moves in the opposite direction as the market, and by a greater magnitude.

and

- $\alpha_i > 0$: The stock has a return in excess of the return we would expect given its risk
- $\alpha_i = 0$: The stock has a return appropriate given its level of risk
- $\alpha_i < 0$: The stock has a return less than what is required given its level of risk

Make sure you install the following libraries in your Anaconda Prompt:

`pip install requests_html` <br />
`pip install yahoo_fin` <br />
`pip install pandas` <br />
`pip install pandas_datareader` <br />
`pip install statsmodels` <br />
`pip install seaborn` <br />

***

## Importing Libraries

The `statsmodels` library is used to run our regression. `pandas` and `pandas_datareader` are used to work with dataframes and import data from the web to dataframes, respectively. The library`datetime` will be used to adjust some dates in our dataframes. Finally, `seaborn` will help us graph our returns.

In [ ]:
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as sm
import pandas as pd
import pandas_datareader.data as web
import datetime
import seaborn as sns

***

## Retrieving Market Data
Ken French of the Tuck School of Business at Darmouth maintains a data library of the daily, weekly, monthly, and annual market risk premiums  $E(r_M) - r_f$, risk free rates $r_f$, and a number of other factors. The `pandas_datareader` allows us to retrieve the data available at http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html. We will use the monthly data and store it in a dataframe.

In [ ]:
#https://pandas-datareader.readthedocs.io/en/latest/readers/famafrench.html
mkt_rf = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench')[0]
mkt_rf.head()

Let's only keep the columns we need and pull the month and year from the date index which we need to do to align the dates with our stock returns dataframe we create later.

In [ ]:
mkt_rf = mkt_rf.drop(['SMB','HML', 'RMW', 'CMA'], axis=1)
mkt_rf = mkt_rf.rename(columns={'Mkt-RF':'mktrf', 'RF':'rf'}) #rename some columns
mkt_rf['month'] = mkt_rf.index.month #create a month column
mkt_rf['year'] = mkt_rf.index.year #create a year column
mkt_rf = mkt_rf.set_index(['year', 'month'])
mkt_rf.head()

***

## Retrieving Stock Data

Now, we obtain stock data from <b><i>Yahoo! Finance</i></b> on a ticker we choose. We'll get data for the last 3 years because the beta as reported in <b><i>Yahoo! Finance</i></b> is the monthly 3 year beta. Again, we create some month and year columns in order to align this data with the monthly data on the market risk premium and risk free rate.


In [ ]:
ticker = input('Input the ticker of the stock: ')
share = get_data(ticker, start_date = '01/21/2016' , end_date = '12/31/2018', index_as_date=True)

share['index1'] = share.index
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html
share['month'] = pd.DatetimeIndex(share['index1']).month #obtain the month
share['year'] = pd.DatetimeIndex(share['index1']).year #obtain the year

share.head()

Now, let's convert the daily stock data to monthly data.

In [ ]:
share_dict = {'open':'first','high':'max','low':'min','close': 'last','volume': 'sum','adjclose': 'last', 'month':'last', 'year': 'last'}
share.index = pd.to_datetime(share.index) #tells pandas that the index is a date
share = share.resample('M', how=share_dict) #take only monthly data as we specified above
share = share.set_index(['year', 'month'])
share.head()

***

## Obtaining Stock Returns

Similar to the previous graphing example, we calculate the stock returns, but here we convert to a percent to match the units of the Ken French data. We use the adjusted close price.


In [ ]:
stock = share['adjclose']
stock_lag = stock.shift(1)
stock_lag.name = 'adjcloselag'
stock = pd.concat([stock, stock_lag], axis=1)
print(stock.head())
stock['return_stock'] = (stock['adjclose'] - stock['adjcloselag'])/stock['adjcloselag']
stock = (stock['return_stock']*100) #Put returns in same units as the returns in the market data file
print(stock.head())

Here we combine the stock returns and the market data and subtract the risk free rate from the returns.

In [ ]:
returns = pd.concat([stock, mkt_rf], axis=1, sort=True) #concatenate the dataframes
returns['r_rf'] = returns['return_stock']-returns['rf']
returns.dropna(inplace=True)

## Run the Regression

In the combined dataframe, we run the regression to obtain beta. This uses the `statsmodels` library we imported.

In [ ]:
#OLS Regression: ttps://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html
result = sm.ols(formula="r_rf ~ mktrf", data = returns).fit()

beta = result.params.mktrf
print (result.summary())
print ('')
print('*********Beta***********')
print (beta)
print('************************')


***

## Plot the Regression Line

Here, we use the `seaborn` library to plot the monthly excess returns of our stock against the monthly excess returns of the market. We can see the graphical representation of the slope ($\beta$) and the intercept ($\alpha$).

In [ ]:
ax = sns.regplot(x="mktrf", y= "r_rf", data=returns )
#ax.set(ylim=(0, 10))
#ax.set(xlim=(0, 10))

***

## The Complete Code

Try out the code from this notebook in Spyder. Use the variable explorer to view the variables and dataframes you create.

In [ ]:
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as sm
import pandas as pd
import pandas_datareader.data as web
import datetime
import seaborn as sns


mkt_rf = web.DataReader('F-F_Research_Data_5_Factors_2x3', 'famafrench')[0]

mkt_rf = mkt_rf.drop(['SMB','HML', 'RMW', 'CMA'], axis=1)
mkt_rf = mkt_rf.rename(columns={'Mkt-RF':'mktrf', 'RF':'rf'}) #rename some columns
mkt_rf['month'] = mkt_rf.index.month #create a month column
mkt_rf['year'] = mkt_rf.index.year #create a year column
mkt_rf = mkt_rf.set_index(['year', 'month'])
mkt_rf.head()

ticker = input('Input the ticker of the stock: ')
share = get_data(ticker, start_date = '01/21/2016' , end_date = '12/31/2018', index_as_date=True)

share['index1'] = share.index
share['month'] = pd.DatetimeIndex(share['index1']).month #obtain the month
share['year'] = pd.DatetimeIndex(share['index1']).year #obtain the year

share_dict = {'open':'first','high':'max','low':'min','close': 'last','volume': 'sum','adjclose': 'last', 'month':'last', 'year': 'last'}
share.index = pd.to_datetime(share.index) #tells pandas that the index is a date
share = share.resample('M', how=share_dict) #take only monthly data as we specified above
share = share.set_index(['year', 'month'])

stock = share['adjclose']
stock_lag = stock.shift(1)
stock_lag.name = 'adjcloselag'
stock = pd.concat([stock, stock_lag], axis=1)
stock['return_stock'] = (stock['adjclose'] - stock['adjcloselag'])/stock['adjcloselag']
stock = (stock['return_stock']*100) #Put returns in same units as the returns in the market data file

returns = pd.concat([stock, mkt_rf], axis=1, sort=True) #concatenate the dataframes
returns['r_rf'] = returns['return_stock']-returns['rf']
returns.dropna(inplace=True)

result = sm.ols(formula="r_rf ~ mktrf", data = returns).fit()

beta = result.params.mktrf
print (result.summary())

ax = sns.regplot(x="mktrf", y= "r_rf", data=returns )
display(ax)
#ax.set(ylim=(0, 10))
#ax.set(xlim=(0, 10))

***

## Further Reading

Be sure to check out the documentation of the libraries we've used. Ken French's data library has additional datasets that you may find useful.

Statsmodels documentation: https://www.statsmodels.org/dev/index.html <br />
Seaborn documentation: https://pypi.org/project/seaborn/ <br />
Ken French's website: http://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html <br />